# Produce movies of our various star terms

In [3]:
import gcsfs
import xarray as xr
import numpy as np
from scale_aware_air_sea.utils import smooth_inputs_dataset, to_zarr_split

In [4]:
fs = gcsfs.GCSFileSystem()

In [5]:
fs.ls('leap-persistent/jbusecke/scale-aware-air-sea/results')
mapper = fs.get_mapper('leap-persistent/jbusecke/scale-aware-air-sea/results/CM26_fluxes_v0.5.zarr')

In [6]:
ds = xr.open_dataset(mapper, engine='zarr', chunks={})
ds

,Array,Chunk
Bytes,5.42 TiB,111.24 MiB
Shape,"(3, 7, 7305, 2700, 3600)","(1, 1, 3, 2700, 3600)"
Count,2 Graph Layers,51135 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,5.42 TiB,111.24 MiB
Shape,"(3, 7, 7305, 2700, 3600)","(1, 1, 3, 2700, 3600)"
Count,2 Graph Layers,51135 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [7]:
# new scale separation
def decomposition(ds):
    def filt(ds):
        return smooth_inputs_dataset(ds, ['yt_ocean', 'xt_ocean'], 50)
    ds_out = {}
    # Q_H (AB) - high resolution input
    ds_out['Q_H'] = ds.sel(smoothing='smooth_full')
    ds_out['Q_H_bar'] = filt(ds_out['Q_H'])
    # Q_L low resolution input
    ds_out['Q_L'] = ds.sel(smoothing='smooth_all')
    ds_out['Q_L_bar'] = filt(ds.sel(smoothing='smooth_all'))
    
    # mixed low resolution input
    ds_out['Q_L_ocean'] = ds.sel(smoothing='smooth_vel_tracer_ocean')
    ds_out['Q_L_ocean_bar'] = filt(ds_out['Q_L_ocean'])
    
    ds_out['Q_L_atmos'] = ds.sel(smoothing='smooth_vel_tracer_atmos')
    ds_out['Q_L_atmos_bar'] = filt(ds_out['Q_L_atmos'])
    
    
    # Inferred Small scale
    ds_out['Q_star'] = ds_out['Q_H_bar'] - ds_out['Q_L']
    ds_out['Q_star_star'] = ds_out['Q_H_bar'] - ds_out['Q_L_bar']
    
    ds_out['Q_star_ocean'] = ds_out['Q_H_bar'] - ds_out['Q_L_ocean']
    ds_out['Q_star_ocean_bar'] = filt(ds_out['Q_star_ocean'])
    ds_out['Q_star_star_ocean'] = ds_out['Q_H_bar'] - ds_out['Q_L_ocean_bar']
    
    ds_out['Q_star_atmos'] = ds_out['Q_H_bar'] - ds_out['Q_L_atmos']
    ds_out['Q_star_atmos_bar'] = filt(ds_out['Q_star_atmos'])
    ds_out['Q_star_star_atmos'] = ds_out['Q_H_bar'] - ds_out['Q_L_atmos_bar']
    
    ds_out['Q_star_res'] = ds_out['Q_star'] - ds_out['Q_star_star_ocean'] - ds_out['Q_star_star_atmos']
    # TODO: rename this. Its an illdefined term (but should correspond to Q_star_star_res + Q_L_prime
    # TODO: Test this actually!
    ds_out['Q_star_star_res'] = ds_out['Q_star_star'] - ds_out['Q_star_star_ocean'] - ds_out['Q_star_star_atmos']
    # ds_out['Q_L_prime'] = 
    
    # for testing
    ds_out['Q_H_bar_bar'] = filt(ds_out['Q_H_bar'])
    ds_out['Q_star_star_star'] = ds_out['Q_H_bar_bar'] - ds_out['Q_L_bar']
    ds_out['Q_star_res_real'] = ds_out['Q_star'] - ds_out['Q_star_ocean'] - ds_out['Q_star_atmos']
    return ds_out

In [8]:
decomp = decomposition(ds)

In [10]:
ds_final.nbytes/1e12

80.945244110228

In [11]:
import dask
dask.config.set(
    {
        "distributed.comm.timeouts.tcp": "720s",
        "distributed.comm.timeouts.connect": "720s",
        "distributed.scheduler.allowed-failures":10,
    }
)

print(dask.config.get("distributed.scheduler.allowed-failures"))

from dask_gateway import Gateway
gateway = Gateway()



# close existing clusters
open_clusters = gateway.list_clusters()
print(list(open_clusters))
if len(open_clusters)>0:
    for c in open_clusters:
        cluster = gateway.connect(c.name)
        cluster.shutdown()  

options = gateway.cluster_options()
options.worker_memory = 52
options.worker_cores = 12

options.environment = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)

# Create a cluster with those options
cluster = gateway.new_cluster(options)
client = cluster.get_client()

# cluster.adapt(10, 200)
cluster.scale(200)
client

10
[]


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.782de30362a74c99930f93da24716ce7/status,


## Write out something I can take a long term average of for Dhruv

In [14]:
ds_write = ds_final.isel(algo=0, time=slice(0,360*5))
ds_write = ds_write.sel(term=['Q_star', 'Q_star_ocean', 'Q_star_atmos','Q_star_res_real','Q_star_res_real', 'Q_star_star', 'Q_star_star_ocean', 'Q_star_star_atmos','Q_star_star_res'])
# Write out temp data
path = 'leap-scratch/jbusecke/scale-aware-air-sea/visualization/CM26_output_global_v2.zarr'
mapper = fs.get_mapper(path)

if fs.exists(path):
    print('Overwriting existing')
    fs.rm(mapper.root, recursive=True)

to_zarr_split(ds_test_write, mapper, split_interval=300)

Overwriting existing
Writing to leap-scratch/jbusecke/scale-aware-air-sea/visualization/CM26_output_global_v2.zarr ...


2022-12-19 20:34:21,090 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('store-map-9c3d14fbafe06d2d6e33778027cf6f17', 16, 45, 0, 0)": ()}


  0%|          | 0/5 [00:00<?, ?it/s]